## OPENAI

In [1]:
import base64
from PIL import Image
def encode_image(image_path):
    # Open the image using PIL
    with Image.open(image_path) as img:
        # Resize the image to 320x320 while maintaining the aspect ratio
        img = img.resize((640, 640))
        
        # Save the resized image to a temporary buffer
        from io import BytesIO
        buffer = BytesIO()
        img.save(buffer, format="JPEG")
        image_data = buffer.getvalue()
    
    # Encode the resized image to base64
    return base64.b64encode(image_data).decode('utf-8')

In [2]:
#user_symptoms="redness, pimples, pustules, swelling, oily skin, visible blood vessels, skin thickening, dryness, itching"
#user_symptoms="rough scaly patch, itching, burning sensation, red spots, open sores, bleeding, crusting, non-healing ulcer, pearly or waxy bump, scar-like area"
user_symptoms="itching, redness, dry skin, thickened skin, scaly patches, inflammation, small raised bumps, oozing, crusting"

In [3]:
prompt = f"""
Based on the image provided, what the skin condition considering the symptoms described: {user_symptoms}. 
The response should include:
1. Name of the possible diseases.
2. Possible causes.
3. Recommended precautions.
4. Suggested medications.

Provide the output in the following JSON format:
{{
  'name': '...',
  'causes': '...',
  'precautions': '...',
  'medications': '...'
}}
Instructions:
1. Only output the JSON, nothing else.
2. Keep in mind the symptopms when analyzing
3. If you think there are 2 diseases then write both
4. If symptoms are of different disease than the image then write both diseases seperated by or.
4. Only return valid JSON
5. Never include backtick symbols such as: `
6. The response will be parsed with json.loads(), therefore it must be valid JSON.
"""

In [4]:
from openai import OpenAI
import os
client = OpenAI()

In [26]:

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image('1.jpeg')}"
                    }
                },
            ],
        }
    ],
    temperature=0.1,
    response_format={type: "json_object"},
)



In [28]:
response=completion.choices[0].message.content

In [31]:
print(type(response))

<class 'str'>


## ALL CODE TOGETHER

In [2]:
import base64
from PIL import Image
#user_symptoms="redness, pimples, pustules, swelling, oily skin, visible blood vessels, skin thickening, dryness, itching"
#user_symptoms="rough scaly patch, itching, burning sensation, red spots, open sores, bleeding, crusting, non-healing ulcer, pearly or waxy bump, scar-like area"
user_symptoms="itching, redness, dry skin, thickened skin, scaly patches, inflammation, small raised bumps, oozing, crusting"
prompt = f"""
Based on the image provided, what the skin condition considering the symptoms described: {user_symptoms}. 
The response should include:
1. Name of the possible diseases.
2. Possible causes.
3. Recommended precautions.
4. Suggested medications.

Provide the output in the following JSON format:
{{
  'name': '...',
  'causes': '...',
  'precautions': '...',
  'medications': '...'
}}
Instructions:
1. Only output the JSON, nothing else.
2. Keep in mind the symptopms when analyzing
3. If you think there are 2 diseases then write both
4. If symptoms are of different disease than the image then write both diseases seperated by or.
4. Only return valid JSON
5. Never include backtick symbols such as: `
6. The response will be parsed with json.loads(), therefore it must be valid JSON.
"""
from openai import OpenAI
import os
client = OpenAI()
def encode_image(image_path):
    # Open the image using PIL
    with Image.open(image_path) as img:
        # Resize the image to 320x320 while maintaining the aspect ratio
        img = img.resize((640, 640))
        
        # Save the resized image to a temporary buffer
        from io import BytesIO
        buffer = BytesIO()
        img.save(buffer, format="JPEG")
        image_data = buffer.getvalue()
    
    # Encode the resized image to base64
    return base64.b64encode(image_data).decode('utf-8')

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image('1.jpeg')}"
                    }
                },
            ],
        }
    ],
    temperature=0.1,
)



BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for response_format 'output_schema': In context=(), object schema missing properties.", 'type': 'invalid_request_error', 'param': 'response_format', 'code': None}}

In [12]:
import json
def find_json_in_string(s):
    decoder = json.JSONDecoder()
    idx = 0
    results = []
    while idx < len(s):
        if s[idx] in '{[':
            try:
                obj, end = decoder.raw_decode(s, idx)
                results.append(obj)
                idx = end
            except json.JSONDecodeError:
                idx += 1
        else:
            idx += 1
    return results

In [19]:
response="""```json
{
    "name": "Acne or Contact Dermatitis",
    "causes": "Acne can be caused by excess oil production, clogged hair follicles, bacteria, and hormones. Contact Dermatitis is caused by exposure to irritants or allergens.",
    "precautions": "For acne, maintain a regular cleansing routine and avoid oily products. For contact dermatitis, identify and avoid the irritant or allergen.",
    "medications": "For acne, consider topical treatments like benzoyl peroxide or salicylic acid. For contact dermatitis, use antihistamines or topical corticosteroids."
}
```"""

In [20]:
result=find_json_in_string(response)

In [21]:
result[0]['name']

'Acne or Contact Dermatitis'